In [ ]:
%matplotlib inline

import essentia.standard as estd
from miran.plots import *
from miran import *
import librosa
import madmom
import numpy as np
import scipy
from sklearn.preprocessing import binarize

In [ ]:
# SELECT A RANDOM TRACK TO ANALYSE
d = '/Users/angel/Insync/datasets/bmtg2/dataset'
f = return_random_track(d, ext=".wav")
show_in_finder(f)
print(f)

In [ ]:
KEY_SETTINGS = {"SAMPLE_RATE": 44100,
                "WINDOW_SIZE": 16384,
                "HOP_SIZE": 2048,
                "WINDOW_SHAPE": "hann",
                "PCP_THRESHOLD": 0,
                "MIN_HZ": 25,
                "MAX_HZ": 3500,
                "SPECTRAL_PEAKS_THRESHOLD": 0.0001,
                "SPECTRAL_PEAKS_MAX": 60,

                "HPCP_BAND_PRESET": False,
                "HPCP_SPLIT_HZ": 250,
                "HPCP_HARMONICS": 4,
                "HPCP_REFERENCE_HZ": 440,
                "HPCP_NON_LINEAR": False,
                "HPCP_NORMALIZE": "unitSum",
                "HPCP_SHIFT": False,
                "HPCP_SIZE": 36,
                "HPCP_WEIGHT_WINDOW_SEMITONES": 3,
                "HPCP_WEIGHT_TYPE": "cosine"}

In [ ]:
import math
25.0 * math.pow(2.,7.)

In [ ]:
3200/np.log2(25)

In [ ]:
# LOAD AUDIO FILE
sig = madmom.audio.signal.Signal(f, sample_rate=KEY_SETTINGS["SAMPLE_RATE"], num_channels=1, dtype="float32")

In [ ]:
cut = estd.FrameCutter(frameSize=KEY_SETTINGS["WINDOW_SIZE"], hopSize=KEY_SETTINGS["HOP_SIZE"])

window = estd.Windowing(size=KEY_SETTINGS["WINDOW_SIZE"])

rfft = estd.Spectrum(size=KEY_SETTINGS["WINDOW_SIZE"])

speaks = estd.SpectralPeaks(magnitudeThreshold=KEY_SETTINGS["SPECTRAL_PEAKS_THRESHOLD"],
                            maxFrequency=KEY_SETTINGS["MAX_HZ"],
                            minFrequency=KEY_SETTINGS["MIN_HZ"],
                            maxPeaks=KEY_SETTINGS["SPECTRAL_PEAKS_MAX"],
                            sampleRate=KEY_SETTINGS["SAMPLE_RATE"])

hpcp = estd.HPCP(bandPreset=KEY_SETTINGS["HPCP_BAND_PRESET"],
                 bandSplitFrequency=KEY_SETTINGS["HPCP_SPLIT_HZ"],
                 harmonics=KEY_SETTINGS["HPCP_HARMONICS"],
                 maxFrequency=KEY_SETTINGS["MAX_HZ"],
                 minFrequency=KEY_SETTINGS["MIN_HZ"],
                 nonLinear=KEY_SETTINGS["HPCP_NON_LINEAR"],
                 normalized=KEY_SETTINGS["HPCP_NORMALIZE"],
                 referenceFrequency=KEY_SETTINGS["HPCP_REFERENCE_HZ"],
                 sampleRate=KEY_SETTINGS["SAMPLE_RATE"],
                 size=KEY_SETTINGS["HPCP_SIZE"],
                 weightType=KEY_SETTINGS["HPCP_WEIGHT_TYPE"],
                 windowSize=KEY_SETTINGS["HPCP_WEIGHT_WINDOW_SEMITONES"],
                 maxShifted=KEY_SETTINGS["HPCP_SHIFT"])

duration = len(sig)
chroma = []

n_slices = int(1 + (duration / KEY_SETTINGS["HOP_SIZE"]))
for slice_n in range(n_slices):
    spek = rfft(window(cut(sig)))
    p1, p2 = speaks(spek)
    #if KEY_SETTINGS["SPECTRAL_WHITENING"]:
    #    p2 = sw(spek, p1, p2)
    pcp = hpcp(p1, p2)
    chroma.append(pcp)
    
    
    
cr_ess = chroma = np.roll(chroma, -3 * (KEY_SETTINGS["HPCP_SIZE"] // 12)).T

plot_chroma(cr_ess, "essentia")

In [ ]:
# use no less than three bins per semitone in order to be able to correct detuning
chroma_os = librosa.feature.chroma_cqt(y=np.array(sig), 
                                       sr=KEY_SETTINGS["SAMPLE_RATE"], 
                                       hop_length=KEY_SETTINGS["HOP_SIZE"], 
                                       fmin=KEY_SETTINGS["MIN_HZ"]
                                       norm=np.inf,
                                       
#                                       n_octaves=
                                       tuning=KEY_SETTINGS["HPCP_REFERENCE_HZ"]
                                       threshold=KEY_SETTINGS["PCP_THRESHOLD"], 
                                       bins_per_octave=KEY_SETTINGS["HPCP_SIZE"],
                                       n_chroma=KEY_SETTINGS["HPCP_SIZE"],
                                       #window=None
                                       cqt_mode="full")

plot_chroma(chroma_os, "librosa")

# add all other parameters!
# librosa.feature.chroma_cqt(C=None,  n_octaves=7,